In [ ]:
import os
import numpy as np
from rail.tools.photometry_tools import LSSTFluxToMagConverter, Dereddener
from rail.utils.catalog_utils import Dc2CatalogConfig
from rail.core.data import TableHandle, DataStore, DATA_STORE
from rail.raruma import utility_functions as raruma_utils

In [ ]:
a_env_dict_in = Dc2CatalogConfig.build_base_dict()['band_a_env']

In [ ]:
DS = DATA_STORE()
DataStore.allow_overwrite = True

In [ ]:
patch = '3951'

In [ ]:
filters = 'ugri'
#flux_types = ['cmodel', 'kron', 'sersic', 'gaap1p0', 'psf', 'gaap3p0']
flux_types = ['cmodel']
a_env_dict = {}
bands = []

for filter_ in filters:
    for flux_type_ in flux_types:
        band = f"{filter_}_{flux_type_}"
        bands.append(band)
        a_env_dict[f"{band}Mag"] = a_env_dict_in[f"mag_{filter_}_cModel_obj_dered"]


In [ ]:
def do_it(patch):
    fl_data = DS.read_file('fl_data', TableHandle, f"{patch}_catalog.pq")
    flux_to_mag = LSSTFluxToMagConverter.make_stage(
        name='flux_to_mag',
        bands=bands,
        flux_name='{band}_flux',
        flux_err_name='{band}_fluxErr',
        mag_name="{band}Mag",
        mag_err_name="{band}MagErr",
        #    copy_cols=dict(redshift='hst_z_spec', ra='coord_best_ra', dec='coord_best_dec'),
        copy_cols=dict(id='id', ra='ra', dec='dec'),
        mag_offset=31.4,
    )
    deredden = Dereddener.make_stage(
        ra_name='ra',
        dec_name='dec',
        mag_name="{band}Mag",
        band_a_env=a_env_dict,
        dustmap_name='sfd',
        dustmap_dir='/Users/echarles/pz/dustmaps',
        copy_all_cols=True,    
    )    
    out_data = flux_to_mag(fl_data)
    flux_to_mag.finalize()
    dered = deredden(out_data)
    deredden.finalize()
    os.system(f"tables-io convert --input output.pq  --output {patch}.hdf5")

In [ ]:
#ll = [
#    '3353',
#    '3354',
#    '3355',
#    '3438',
#    '3439',
#    '3440',
#    '3523',
#    '3524',
#    '3525',
#]
ll = ['3951']

In [ ]:
for v in ll:
   do_it(v)

In [ ]:
#fl_data = DS.read_file('fl_data', TableHandle, '/Users/echarles/pz/sandbox_data/cdfs_matched_hst.pq')
fl_data = DS.read_file('fl_data', TableHandle, f"{patch}_catalog.pq")

In [ ]:
fl_data.data

In [ ]:
a_env_dict.keys()

In [ ]:
flux_to_mag = LSSTFluxToMagConverter.make_stage(
    name='flux_to_mag',
    bands=bands,
    flux_name='{band}_flux',
    flux_err_name='{band}_fluxErr',
    mag_name="{band}Mag",
    mag_err_name="{band}MagErr",
#    copy_cols=dict(redshift='hst_z_spec', ra='coord_best_ra', dec='coord_best_dec'),
    copy_cols=dict(id='id', ra='ra', dec='dec'),
    mag_offset=31.4,
)

In [ ]:
deredden = Dereddener.make_stage(
    ra_name='ra',
    dec_name='dec',
    mag_name="{band}Mag",
    band_a_env=a_env_dict,
    dustmap_name='sfd',
    dustmap_dir='/Users/echarles/pz/dustmaps',
    copy_all_cols=True,    
)

In [ ]:
out_data = flux_to_mag(fl_data)

In [ ]:
flux_to_mag.finalize()

In [ ]:
dered = deredden(out_data)

In [ ]:
deredden.finalize()

In [ ]:
os.system(f"tables-io convert --input output.pq  --output {patch}.hdf5")